# Setup

In [1]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
import json
import re
import os
import warnings
import openmeteo_requests
import great_expectations as ge
warnings.filterwarnings("ignore")

In [2]:
with open("data/hopsworks-token.txt") as f:
    hopsworks_api_key = f.read().rstrip()
with open("data/aqi-token.txt") as f:
    aqi_token = f.read().rstrip()

In [3]:
csv_file = "data/reutlingen.csv"
country = "germany"
city = "reutlingen"
street = "zaisentalstraße"
aqicn_url = "https://api.waqi.info/feed/A54451"

# Historical air quality

In [4]:
df = pd.read_csv(csv_file, parse_dates=["date"], skipinitialspace=True)
air_quality_df = df[["date", "pm25"]]
air_quality_df["country"] = country
air_quality_df["city"] = city
air_quality_df["street"] = street
air_quality_df["url"] = aqicn_url
air_quality_df.dropna(inplace=True)
air_quality_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1785 entries, 0 to 1784
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype              
---  ------   --------------  -----              
 0   date     1785 non-null   datetime64[ns, UTC]
 1   pm25     1785 non-null   float64            
 2   country  1785 non-null   object             
 3   city     1785 non-null   object             
 4   street   1785 non-null   object             
 5   url      1785 non-null   object             
dtypes: datetime64[ns, UTC](1), float64(1), object(4)
memory usage: 83.8+ KB


# Historical weather

In [5]:
om = openmeteo_requests.Client()
openmeteo_url = "https://archive-api.open-meteo.com/v1/archive"
latitude = 48.50052694662806
longitude = 9.194111640132975
today = datetime.date.today()

In [6]:
earliest_date = pd.Series.min(air_quality_df['date'])
earliest_date = earliest_date.strftime('%Y-%m-%d')
earliest_date

'2019-12-09'

In [7]:
params = {
    "latitude": latitude,
    "longitude": longitude,
    "start_date": earliest_date,
    "end_date": today,
    "daily": ["temperature_2m_mean", "precipitation_sum", "wind_speed_10m_max", "wind_direction_10m_dominant"]
}
responses = om.weather_api(openmeteo_url, params=params)
response = responses[0]
# Sanity check, taken from repository.
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 48.47100067138672°N 9.075630187988281°E
Elevation 378.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [8]:
# Taken from the book repository.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(1).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(2).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(3).ValuesAsNumpy()

# Create data frame, fill it.
daily_data = {"date": pd.date_range(
    start=pd.to_datetime(daily.Time(), unit="s"),
    end=pd.to_datetime(daily.TimeEnd(), unit="s"),
    freq=pd.Timedelta(seconds=daily.Interval()),
    inclusive="left"
    ), "temperature_2m_mean": daily_temperature_2m_mean, "precipitation_sum": daily_precipitation_sum,
    "wind_speed_10m_max": daily_wind_speed_10m_max, "wind_direction_10m_dominant": daily_wind_direction_10m_dominant}

daily_dataframe = pd.DataFrame(data = daily_data)
daily_dataframe = daily_dataframe.dropna()
daily_dataframe['city'] = city
daily_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1793 entries, 0 to 1792
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         1793 non-null   datetime64[ns]
 1   temperature_2m_mean          1793 non-null   float32       
 2   precipitation_sum            1793 non-null   float32       
 3   wind_speed_10m_max           1793 non-null   float32       
 4   wind_direction_10m_dominant  1793 non-null   float32       
 5   city                         1793 non-null   object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 70.0+ KB


# Validation

In [9]:
aq_expect = ge.core.ExpectationSuite(expectation_suite_name="aq_expect")
aq_expect.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"pm25",
            "min_value":-0.1,
            "max_value":500.0,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "pm25", "min_value": -0.1, "max_value": 500.0, "strict_min": true}, "meta": {}}

In [10]:
weather_expect = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

def expect_greater_than_zero(column):
    weather_expect.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column":column,
                "min_value":-0.1,
                "max_value":1000.0,
                "strict_min":True
            }
        )
    )
expect_greater_than_zero("precipitation_sum")
expect_greater_than_zero("wind_speed_10m_max")

# Hopsworks feature store

In [11]:
project = hopsworks.login(api_key_value=hopsworks_api_key)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1160342


In [12]:
secrets = hopsworks.connection(host="c.app.hopsworks.ai", api_key_value=hopsworks_api_key).get_secrets_api()

Connected. Call `.close()` to terminate connection gracefully.


In [19]:
try:
    secrets.create_secret("AQI_API_KEY", aqi_token)
except hopsworks.RestAPIError:
    AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value

In [20]:
dict_obj = {
    "country": country,
    "city": city,
    "street": street,
    "aqicn_url": aqicn_url,
    "latitude": latitude,
    "longitude": longitude
}
str_dict = json.dumps(dict_obj)
str_dict

'{"country": "germany", "city": "reutlingen", "street": "zaisentalstra\\u00dfe", "aqicn_url": "https://api.waqi.info/feed/A54451", "latitude": 48.50052694662806, "longitude": 9.194111640132975}'

In [21]:
try:
    secrets.create_secret("SENSOR_LOCATION_JSON", str_dict)
except hopsworks.RestAPIError:
    print("Already exists, please delete from UI")
    existing_key = secrets.get_secret("SENSOR_LOCATION_JSON").value
    print(f"{existing_key}")

Already exists, please delete from UI
{"country": "germany", "city": "reutlingen", "street": "zaisentalstra\u00dfe", "aqicn_url": "https://api.waqi.info/feed/A54451", "latitude": 48.50052694662806, "longitude": 9.194111640132975}


In [22]:
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [25]:
air_quality_fg = fs.get_or_create_feature_group(
    name='air_quality',
    description='Air Quality characteristics of each day',
    version=1,
    primary_key=['city', 'street', 'date'],
    event_time="date",
    expectation_suite=aq_expect
)

In [26]:
air_quality_fg.insert(air_quality_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1160342/fs/1151045/fg/1338721
2024-11-06 16:44:05,900 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1160342/fs/1151045/fg/1338721


Uploading Dataframe: 0.00% |          | Rows 0/1785 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1160342/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x28fe4b290>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 662535
         }
       },
       "result": {
         "observed_value": 0.55,
         "element_count": 1785,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-06T03:44:05.000900Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "successful_expectations": 1,
     "uns

In [27]:
air_quality_fg.update_feature_description("date", "Date of measurement of air quality")
air_quality_fg.update_feature_description("country", "Country where the air quality was measured (sometimes a city in acqcn.org)")
air_quality_fg.update_feature_description("city", "City where the air quality was measured")
air_quality_fg.update_feature_description("street", "Street in the city where the air quality was measured")
air_quality_fg.update_feature_description("pm25", "Particles less than 2.5 micrometers in diameter (fine particles) pose health risk")

In [28]:
weather_fg = fs.get_or_create_feature_group(
    name='weather',
    description='Weather characteristics of each day',
    version=1,
    primary_key=['city', 'date'],
    event_time="date",
    expectation_suite=weather_expect
)

In [29]:
weather_fg.insert(daily_dataframe)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1160342/fs/1151045/fg/1337696
2024-11-06 16:44:25,296 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1160342/fs/1151045/fg/1337696


Uploading Dataframe: 0.00% |          | Rows 0/1793 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1160342/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x17e2ae6c0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 661512
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 1793,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-06T03:44:25.000295Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_betwe

In [30]:
weather_fg.update_feature_description("date", "Date of measurement of weather")
weather_fg.update_feature_description("city", "City where weather is measured/forecast for")
weather_fg.update_feature_description("temperature_2m_mean", "Temperature in Celsius")
weather_fg.update_feature_description("precipitation_sum", "Precipitation (rain/snow) in mm")
weather_fg.update_feature_description("wind_speed_10m_max", "Wind speed at 10m above ground")
weather_fg.update_feature_description("wind_direction_10m_dominant", "Dominant Wind direction over the days")